In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
import numpy as np

## Load Pickles of checkpoint charges per byte

In [3]:
from os import path, pardir, scandir
import sys
import pickle
from math import ceil

In [4]:
from pydgilib_extra import LOGGER_CSV

In [5]:
sys.path.insert(0,'../..')

In [6]:
measure_security = False
show_lm_plot=2
measure_workloads = True
repetitions = 100

In [7]:
from experiments.checkpoint_energy import CheckpointEnergy

In [8]:
text_as_size = False

security_method = 'No Security'

#checkpoint_energy = CheckpointEnergy()
# checkpoint_energy = CheckpointEnergy(security_projects=['AES_Flash', 'TrustZone_Flash', 'No_Security_Flash'])
checkpoint_energy = CheckpointEnergy(workload_folder = "../../Workloads"
                                     ,security_projects=['No-Security_O1_4Mhz',
                                                        'No-Security_O1_12Mhz',
                                                        'No-Security_O3_4Mhz',
                                                        'No-Security_O3_12Mhz'], text_as_size=text_as_size)

In [9]:
checkpoint_energy.security_projects

['No-Security_O1_4Mhz',
 'No-Security_O1_12Mhz',
 'No-Security_O3_4Mhz',
 'No-Security_O3_12Mhz']

In [10]:
checkpoint_energy.workload_projects

['BitCounting', 'Dijkstra', 'FFT', 'Sorting']

### Smoothen?

Get lookup function

In [11]:
get_security_energy = checkpoint_energy.get_security_energy_function()

In [12]:
for security_type in checkpoint_energy.security_projects + ["None"]:
    print(security_type, get_security_energy(security_type,2048))
    print(security_type, get_security_energy(security_type,2048, use_model=True))
    print(security_type, get_security_energy(security_type,100000000))

No-Security_O1_4Mhz {'No Security Flash Write': 7.83852588068152e-05, 'No Security Flash Read': 2.0069835765135478e-06}
No-Security_O1_4Mhz {'No Security Flash Write': 7.404323271431247e-05, 'No Security Flash Read': 1.9974966075267753e-06}
No-Security_O1_4Mhz {'No Security Flash Write': 3.4201679425044835, 'No Security Flash Read': 0.09958481777066054}
No-Security_O1_12Mhz {'No Security Flash Write': 9.235259321160087e-05, 'No Security Flash Read': 1.7979811016691636e-06}
No-Security_O1_12Mhz {'No Security Flash Write': 8.727605325690089e-05, 'No Security Flash Read': 1.7988133545383578e-06}
No-Security_O1_12Mhz {'No Security Flash Write': 3.9779628152536315, 'No Security Flash Read': 0.08829428061043977}
No-Security_O3_4Mhz {'No Security Flash Write': 7.706895844451065e-05, 'No Security Flash Read': 1.8923870373786442e-06}
No-Security_O3_4Mhz {'No Security Flash Write': 7.272500303049978e-05, 'No Security Flash Read': 1.8803323629722353e-06}
No-Security_O3_4Mhz {'No Security Flash Wr

In [13]:
import numpy as np
from math import sqrt, floor
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.ticker as tck


In [14]:
c_scale = 1e3
j_scale = 1e6
t_scale = 1e3
voltage = 3.31

In [15]:
checkpoint_energy.security_projects

['No-Security_O1_4Mhz',
 'No-Security_O1_12Mhz',
 'No-Security_O3_4Mhz',
 'No-Security_O3_12Mhz']

In [16]:
workload_colors = {'BitCounting':'tab:blue', 'Dijkstra':'tab:green', 'FFT':'tab:red', 'Sorting':'tab:orange'}
# security_colors = {'AES_Flash':'#85e085', 'TrustZone_Flash':'#66b3ff', 'No_Security_Flash':'#b3b3cc'}
security_colors = {'No-Security_O1_4Mhz':'#b3b3cc', 
                   'No-Security_O1_12Mhz':'#b3b3cc',
                   'No-Security_O3_4Mhz':'#b3b3cc',
                   'No-Security_O3_12Mhz':'#b3b3cc'}
colors = {**workload_colors, **security_colors}

In [17]:
security_projects=[ 'No-Security_O1_4Mhz',
                                                        'No-Security_O1_12Mhz',
                                                        'No-Security_O3_4Mhz',
                                                        'No-Security_O3_12Mhz']

In [18]:
hatches = ['-', '+', 'x', '\\', '*', 'o', 'O', '.']

In [19]:
section_hatches = {'AES Encrypt': '\\\\', 
                   'AES Decrypt': '//', 
                   'AES-256 Encrypt': '\\\\', 
                   'AES-256 Decrypt': '//', 
                   'AES-128 Encrypt': '\\\\', 
                   'AES-128 Decrypt': '//', 
                   'Flash Write': 'xx', 
                   'Flash Read': '/////', 
                   'AES-256 Flash Write': 'xx', 
                   'AES-256 Flash Read': '/////', 
                   'AES-128 Flash Write': 'xx', 
                   'AES-128 Flash Read': '/////', 
                   'No Security Flash Write': 'xx', 
                   'No Security Flash Read': '/////', 
                   'TrustZone Flash Write': 'xx', 
                   'TrustZone Flash Read': '/////', 
                   'Bit Counting': 'ooo', 
                   'Create Graph': '0', 
                   'Dijkstra': 'ooo',
                   'Allocate FFT': '0',
                   'FFT': 'ooo',
                   'Free FFT': '...',
                   'Sorting': 'ooo'
                  }


circ1 = mpatches.Patch( facecolor='None', hatch=r'\\\\',label='AES CBC Mode Encrypt')
circ2 = mpatches.Patch( facecolor='None', hatch='//',label='AES CBC Mode Decrypt')
circ3 = mpatches.Patch( facecolor='None', hatch='xx',label='Flash Write')
circ4 = mpatches.Patch( facecolor='None', hatch='/////',label='Flash Read')
        

In [20]:
edgecolor = 'black'
linewidth = 0.8

In [21]:
xLabels = ['O1\n4MHz','O1\n12MHz','O3\n4MHz',
            'O3\n12MHz']

In [22]:
#security_projects=['AES-128_Flash', 'AES-256_Flash', 'TrustZone_Flash', 'No_Security_Flash']

In [23]:
totalEnergy = get_security_energy('No-Security_O1_4Mhz',4096, use_model=True)
sum =0
for evl in totalEnergy.values():
    sum+=evl*j_scale*voltage
sum    

490.2943617799731

In [24]:
class WorkloadPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(workload_folder = "../../Workloads",security_projects=security_projects), fig=None, ax=None, show=False):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        if show:
            self.make_fig()
            self.update_fig()
            
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
    
    def update_fig(self):
        opacity = 0.8
        self.ax.cla()
        
        self.bars = {}
        for index, workload_project in enumerate(self.checkpoint_energy.workload_projects):
            self.bars[workload_project] = {}
            section_averages = self.checkpoint_energy.get_workload_average(workload_project)
            section_stds = self.checkpoint_energy.get_workload_std(workload_project)
            bottom = 0
            for section in section_averages.keys():
                self.bars[workload_project][section] = self.ax.bar(index, section_averages[section]*j_scale*voltage, color=colors[workload_project], 
                                                                   label=section, alpha=opacity, yerr=section_stds[section]*j_scale*voltage, bottom=bottom, 
                                                                   hatch=section_hatches[section], linewidth=linewidth, edgecolor=edgecolor)
                bottom += section_averages[section]*j_scale*voltage
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.workload_projects)))
        self.ax.set_xticklabels(self.checkpoint_energy.workload_projects)
        self.ax.legend()
#         self.ax.set_yscale('log')
        # self.ax.grid()

        self.ax.set_xlabel('Workload')
        self.ax.set_ylabel('Energy [$\mu$J]')
        self.ax.set_title('Energy per Section')
        self.fig.canvas.set_window_title('Energy per Workload')
        self.fig.show()

In [25]:
workload_plotter = WorkloadPlotter(checkpoint_energy, show=True)

In [26]:
# workload_plotter.make_fig()

In [27]:
# workload_plotter.update_fig()

In [28]:
checkpoint_energy.security_projects
#checkpoint_energy.get_security_energy('AES-256_Crypto-Accelerator', 4096)

['No-Security_O1_4Mhz',
 'No-Security_O1_12Mhz',
 'No-Security_O3_4Mhz',
 'No-Security_O3_12Mhz']

In [29]:
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)


In [30]:
def set_xmargin(ax, left=0.0, right=0.3):
    ax.set_xmargin(0)
    ax.autoscale_view()
    lim = ax.get_xlim()
    delta = np.diff(lim)
    left = lim[0] - delta*left
    right = lim[1] + delta*right
    ax.set_xlim(left,right)

In [31]:
class SecurityPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(workload_folder = "../../Workloads",security_projects=security_projects), num_bytes=4096, fig=None, ax=None, show=0, use_model=False):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        self.num_bytes = num_bytes
        self.use_model = use_model
        if show:
            self.make_fig(figsize=(12.8, 6.8),dpi= 300) #width, height in inches
            self.update_fig()
            if show >= 2:
                self.animate()
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
    
    def update_fig(self, num_bytes=None):
        if num_bytes is None:
            num_bytes = self.num_bytes
        opacity = 0.8
        self.ax.cla()
#         self.ax.grid(zorder=0)
        self.ax.yaxis.grid(True, alpha=0.3)
        
        self.bars = {}
        for index, security_project in enumerate(self.checkpoint_energy.security_projects):
            self.bars[security_project] = {}
            security_energy = self.checkpoint_energy.get_security_energy(security_project, num_bytes, use_model=self.use_model)
            #print(f'{security_energy}')
            bottom = 0
            sectionEnergies = get_security_energy(security_project,num_bytes, use_model=self.use_model)
            totalEnergy = 0
            for sectionEnergy in sectionEnergies.values():
                totalEnergy+=sectionEnergy * j_scale * voltage
            #print(f'{totalEnergy}')
            for section in security_energy.keys():
                self.bars[security_project][section] = self.ax.bar(index, security_energy[section] * j_scale*voltage, color=colors[security_project], 
                                                                   label=section, width=0.4, alpha=opacity, bottom=bottom, zorder=3,
                                                                   hatch=section_hatches[section], linewidth=linewidth, edgecolor=edgecolor)
                sectionValue = security_energy[section] * j_scale*voltage
                accumulatedValue = sectionValue + bottom
                yValue = (totalEnergy - accumulatedValue/2)
                bar = self.bars[security_project][section].get_children()
                print(f'{index} : {sectionValue}')
                self.ax.text(x=index + (bar[0].get_width()*1.01 ) , y= bottom + sectionValue/2 -15 , 
                             s=f"{round(sectionValue)}", ha = 'center', va = 'center', fontdict=dict(fontsize=11))
                bottom += security_energy[section]*j_scale*voltage
                
            self.ax.text(x=index + 0.03 , y =bottom+25 , s=f"{round(bottom, 2)}" ,bbox=dict(boxstyle="square, pad=0.03", fc="none", ec="gray", zorder=0.9, lw=0.5),
                         va='baseline', horizontalalignment='center', fontdict=dict(fontsize=12))
    
            print(f'{security_project}: {bottom}')        
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.security_projects)) )
     ##   self.ax.set_xticklabels([s.replace('_',' ') for s in self.checkpoint_energy.security_projects])
        self.ax.set_xticklabels(xLabels, fontdict=dict(fontsize=12))
        self.ax.yaxis.set_tick_params(labelsize=12)

     #   self.ax.set_xticklabels([" ".join(s.split('_')[:-1]) for s in self.checkpoint_energy.security_projects])
       
      #  self.ax.set_yticks( fontdict=dict(fontsize=4))
      #  self.ax.legend(handles = [circ1,circ2,circ3, circ4],loc=6, prop={'size': 4})
       
      #  self.ax.set_yscale('log')

       # self.ax.set_xlabel('\nDifferent AES-Modes execution on AES-Module')
        self.ax.set_ylabel('Energy [$\mu$J]', fontdict=dict(fontsize=12))
        self.ax.set_title(f'(d) No Security' ,  fontdict=dict(fontsize=12))
        self.fig.canvas.set_window_title('Energy_per_parameter_mbedTLS')
        self.ax.margins( y= 0.12)
        self.ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

       # self.ax.set(adjustable="datalim")
        #ratio = 0.5
        #xleft, xright = self.ax.get_xlim()
        #ybottom, ytop = self.ax.get_ylim()
        # the abs method is used to make sure that all numbers are positive
        # because x and y axis of an axes maybe inversed.
        #self.ax.set_aspect(abs((xright-xleft)/(ybottom-ytop))*ratio)
        
        set_xmargin(self.ax, left=0.05, right=0.11)
        #set_size(6.4,12.4, self.ax)
        plt.gcf().set_size_inches(5, 7)

        self.fig.tight_layout()
        
        self.fig.savefig('Energy_per_parameter_NoSecurity.pdf', bbox_inches='tight', dpi=1000)
        self.fig.savefig('Energy_per_parameter_NoSecurity.svg', bbox_inches='tight', dpi=300)
        self.fig.show()
    def anim_func(self, num_bytes, *fargs):
        self.update_fig(num_bytes)
        return flatten(self.bars)
    
    def animate(self, frames=range(100, 6001, 100), save=False, *save_args, **save_kwargs):
        self.animation = animation.FuncAnimation(self.fig, self.anim_func, blit=True, interval=0,frames=frames ,repeat=False)
        if save:
            self.animation.save('Charge_per_Security.mp4', *save_args, **save_kwargs)
        
        
def flatten(d, ret=None):
    if ret is None:
        ret = []
    for k, v in sorted(d.items()):
        if isinstance(v, dict):
            flatten(v, ret)
        else:
            ret.extend(v.get_children())
    return ret

In [32]:
security_plotter = SecurityPlotter(checkpoint_energy, show=1, use_model=False)
#security_plotter = SecurityPlotter(checkpoint_energy, show=1, use_model=True)

0 : 489.99661575827
0 : 13.2985848543289
No-Security_O1_4Mhz: 503.29520061259893
1 : 574.3692648075906
1 : 11.98737734000716
No-Security_O1_12Mhz: 586.3566421475978
2 : 484.00548634960876
2 : 12.592531692503098
No-Security_O3_4Mhz: 496.59801804211185
3 : 574.2244704298703
3 : 11.941046262892982
No-Security_O3_12Mhz: 586.1655166927633


## Security Time

In [33]:
def get_security_time(project_name, num_bytes):
    time_dict = {}
    project_path = [".", project_name]
    #encrypt_time_ms = pickle.load(open(path.join(*project_path, "aes_flash_encrypt_time_ms.p"), "rb"))
    #decrypt_time_ms = pickle.load(open(path.join(*project_path, "aes_flash_decrypt_time_ms.p"), "rb"))
    flash_write_time_ms = pickle.load(open(path.join(*project_path, "no_security_flash_write_time_ms.p"), "rb"))
    flash_read_time_ms = pickle.load(open(path.join(*project_path, "no_security_flash_read_time_ms.p"), "rb"))
    
    i = round((num_bytes) - 1)
    print(f'{i}')
    #time_dict['AES-256 Encrypt'] = encrypt_time_ms[i]
    #time_dict['AES-256 Decrypt'] = decrypt_time_ms[i]
    time_dict['AES-256 Flash Write'] =  flash_write_time_ms[i]
    time_dict['AES-256 Flash Read'] = flash_read_time_ms[i]
    
    return time_dict

In [34]:
class SecurityPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(workload_folder = "../../Workloads",security_projects=security_projects), num_bytes=4096, fig=None, ax=None, show=0, use_model=False):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        self.num_bytes = num_bytes
        self.use_model = use_model
        if show:
            self.make_fig(figsize=(12.8, 6.8),dpi= 300) #width, height in inches
            self.update_fig()
            if show >= 2:
                self.animate()
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
    
    def update_fig(self, num_bytes=None):
        if num_bytes is None:
            num_bytes = self.num_bytes
        opacity = 0.8
        self.ax.cla()
#         self.ax.grid(zorder=0)
        self.ax.yaxis.grid(True, alpha=0.3)
        
        self.bars = {}
        for index, security_project in enumerate(self.checkpoint_energy.security_projects):
            self.bars[security_project] = {}
            security_time = get_security_time(security_project, num_bytes)
            #print(f'{security_energy}')
            bottom = 0
            sectionTimes = get_security_time(security_project,num_bytes)
            totalTime = 0
            for sectionTime in sectionTimes.values():
                totalTime+=sectionTime
            #print(f'{totalEnergy}')
            for section in security_time.keys():
                self.bars[security_project][section] = self.ax.bar(index, security_time[section] , color=colors[security_project], 
                                                                   label=section, width=0.4, alpha=opacity, bottom=bottom, zorder=3,
                                                                   hatch=section_hatches[section], linewidth=linewidth, edgecolor=edgecolor)
                sectionValue = security_time[section] 
                accumulatedValue = sectionValue + bottom
                yValue = (totalTime - accumulatedValue/2)
                bar = self.bars[security_project][section].get_children()
                print(f'{index} : {sectionValue}')
                self.ax.text(x=index + (bar[0].get_width()*0.85 ) , y= bottom + sectionValue/3  , 
                             s=f"{round(sectionValue)}", ha = 'center', va = 'center', fontdict=dict(fontsize=11))
                bottom += security_time[section]
                
            self.ax.text(x=index + 0.01 , y =bottom+4 , s=f"{round(bottom, 2)}" ,bbox=dict(boxstyle="square, pad=0.03", fc="none", ec="gray", zorder=0.9, lw=0.5),
                         va='baseline', horizontalalignment='center', fontdict=dict(fontsize=12))
    
            print(f'{security_project}: {bottom}')        
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.security_projects)) )
     ##   self.ax.set_xticklabels([s.replace('_',' ') for s in self.checkpoint_energy.security_projects])
        self.ax.set_xticklabels(xLabels, fontdict=dict(fontsize=12))
        self.ax.yaxis.set_tick_params(labelsize=12)

     #   self.ax.set_xticklabels([" ".join(s.split('_')[:-1]) for s in self.checkpoint_energy.security_projects])
       
      #  self.ax.set_yticks( fontdict=dict(fontsize=4))
      #  self.ax.legend(handles = [circ1,circ2,circ3, circ4],loc=6, prop={'size': 4})
       
      #  self.ax.set_yscale('log')

       # self.ax.set_xlabel('\nDifferent AES-Modes execution on AES-Module')
        self.ax.set_ylabel('Time [ms]', fontdict=dict(fontsize=12))
        self.ax.set_title(f'(d) {security_method}' ,  fontdict=dict(fontsize=12))
        self.fig.canvas.set_window_title(f'Time_per_parameter_{security_method}')
        self.ax.margins( y= 0.12)
        self.ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

       # self.ax.set(adjustable="datalim")
        #ratio = 0.5
        #xleft, xright = self.ax.get_xlim()
        #ybottom, ytop = self.ax.get_ylim()
        # the abs method is used to make sure that all numbers are positive
        # because x and y axis of an axes maybe inversed.
        #self.ax.set_aspect(abs((xright-xleft)/(ybottom-ytop))*ratio)
        
        set_xmargin(self.ax, left=0.05, right=0.11)
        #set_size(6.4,12.4, self.ax)
        plt.gcf().set_size_inches(5, 7)

        self.fig.tight_layout()
        
        self.fig.savefig(f'Time_per_parameter_NoSecurity.pdf', bbox_inches='tight', dpi=1000)
        self.fig.savefig(f'Time_per_parameter_NoSecurity.svg', bbox_inches='tight', dpi=1000)
        self.fig.show()
    def anim_func(self, num_bytes, *fargs):
        self.update_fig(num_bytes)
        return flatten(self.bars)
    
    def animate(self, frames=range(100, 6001, 100), save=False, *save_args, **save_kwargs):
        self.animation = animation.FuncAnimation(self.fig, self.anim_func, blit=True, interval=0,frames=frames ,repeat=False)
        if save:
            self.animation.save('Charge_per_Security.mp4', *save_args, **save_kwargs)
        
        
def flatten(d, ret=None):
    if ret is None:
        ret = []
    for k, v in sorted(d.items()):
        if isinstance(v, dict):
            flatten(v, ret)
        else:
            ret.extend(v.get_children())
    return ret

In [35]:
#Security Time Pass same parameters
security_plotter = SecurityPlotter(checkpoint_energy, show=1, use_model=False)

4095
4095
0 : 88.37305013338437
0 : 8.77856746666339
No-Security_O1_4Mhz: 97.15161760004776
4095
4095
1 : 58.476484266691386
1 : 2.9039061333264726
No-Security_O1_12Mhz: 61.38039040001786
4095
4095
2 : 82.37145813330926
2 : 8.249666666642952
No-Security_O3_4Mhz: 90.62112479995221
4095
4095
3 : 58.803438933352936
3 : 2.903859199989256
No-Security_O3_12Mhz: 61.70729813334219
